In [7]:
#imports
#pip install scikit-learn
#pip install pandas

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn import metrics
# We need to do some web scraping to get the review text
import time
import urllib.request
from bs4 import BeautifulSoup

In [2]:
#load in datasets
GENERAL_DATASET_PATH = "data/general_reviews.csv"
ELDEN_RING_DATASET_PATH = "data/elden_ring_steam_reviews.csv"
general_data = pd.read_csv(GENERAL_DATASET_PATH)
elden_ring_data = pd.read_csv(ELDEN_RING_DATASET_PATH)
general_data.head()

,Gaming Console name,Review link,Review Title,Time Since Review,Comment Count,Likes Received,Rating/10,Rating Comments
0,NS,https://www.gamespot.com/reviews/tales-of-symp...,Tales Of Symphonia Remastered Review - A Class...,20 days ago,23,6,5.0,Mediocre
1,NS,https://www.gamespot.com/reviews/kirbys-return...,Kirby's Return To Dream Land Deluxe Review - K...,22 days ago,2,2,7.0,Good
2,PS5,https://www.gamespot.com/reviews/hogwarts-lega...,Hogwarts Legacy Review - Sleight Of Hand,23 days ago,0,593,6.0,Fair
3,XBSX,https://www.gamespot.com/reviews/atomic-heart-...,Atomic Heart Review - Crispy Critters,23 days ago,226,23,6.0,Fair
4,PC,https://www.gamespot.com/reviews/like-a-dragon...,Like A Dragon: Ishin Review - Rewriting History,27 days ago,11,4,8.0,Great


In [3]:
# Clean and format Data
# Lets only keep the columns we want
general_data = general_data[['Review link','Rating/10','Rating Comments']]
general_data.head()

,Review link,Rating/10,Rating Comments
0,https://www.gamespot.com/reviews/tales-of-symp...,5.0,Mediocre
1,https://www.gamespot.com/reviews/kirbys-return...,7.0,Good
2,https://www.gamespot.com/reviews/hogwarts-lega...,6.0,Fair
3,https://www.gamespot.com/reviews/atomic-heart-...,6.0,Fair
4,https://www.gamespot.com/reviews/like-a-dragon...,8.0,Great


In [4]:
# We can get all the comment types with:
# print(general_data['Rating Comments'].unique())
# This gives the following list:
# ['Mediocre' 'Good' 'Fair' 'Great' 'Superb' 'Poor' 'Essential' 'Abysmal'
#  'Early Access' 'Terrible' 'Bad' 'Good\x1a' nan 'Essential"width-100 src']
# Lets convert these into just good and bad. We will make a dictionary showing what to convert to in each case
GOOD = 0
NUETRAL = 1
BAD = 2
#In case we want to use rating to indicate sentiment
# def GetSentiment(rating):
#     if(rating < 5):
#         return BAD
#     else:
#         return GOOD
comment_dict = {'Mediocre':NUETRAL,'Good':GOOD,'Fair':NUETRAL,'Great':GOOD,'Superb':GOOD,
                'Poor':BAD,'Essential':GOOD,'Abysmal':BAD,'Early Access':NUETRAL,
                'Terrible':BAD,'Bad':BAD,'Good\x1a':GOOD,'Essential"width-100 src':None}
#Convert rating comments using dict
general_data.replace({'Rating Comments':comment_dict}, inplace=True)
general_data.dropna(inplace=True)
print(general_data['Rating Comments'].unique())
print(f"shape={general_data.shape}")
#Lets rename Rating Comments to Sentiment
general_data.rename(columns={'Rating Comments':'Sentiment'},inplace=True)
general_data.head()

[1. 0. 2.]
shape=(12591, 3)


,Review link,Rating/10,Sentiment
0,https://www.gamespot.com/reviews/tales-of-symp...,5.0,1.0
1,https://www.gamespot.com/reviews/kirbys-return...,7.0,0.0
2,https://www.gamespot.com/reviews/hogwarts-lega...,6.0,1.0
3,https://www.gamespot.com/reviews/atomic-heart-...,6.0,1.0
4,https://www.gamespot.com/reviews/like-a-dragon...,8.0,0.0


In [ ]:
# Now lets go get the text of the reviews
i = 0
num_links = len(general_data.index)
check_interval = 5
reviews = []
for link in general_data['Review link']:
    if(i % check_interval == 0):
        if(i>0):
            prediction = (time.time() - prev_time)/(check_interval) * (num_links/len(reviews))
            print(f"processed: {i} / {num_links}")
            print(f"Time left: {int(prediction)} s")  
        prev_time = time.time()
    i = i + 1
    try:
        page = urllib.request.urlopen(link).read()
        page = BeautifulSoup(page)
        review = ""
        body = page.find(class_="content-body review-article article article--review article--one-column content-body--buffer")
        body = body.find(class_="article-body typography-format")
        paragraphs = body.find_all("p",dir="ltr")
        for p in paragraphs:
            review+=p.text+" "
        reviews.append(review)
    except:
        reviews.append(None)
general_data['Review'] = reviews

processed: 0
processed: 50
Time left: 237.5539359998703 s
processed: 100
Time left: 104.44958721213341 s
processed: 150
Time left: 72.27793391332625 s
processed: 200
Time left: 55.81909035143852 s
processed: 250
Time left: 46.37267475797653 s
processed: 300
Time left: 35.65083980154991 s


In [ ]:
general_data.to_csv('general_data_clean.csv',index=False)

In [ ]:
#split datasets
TEST_SIZE = 0.3
general_X = 
general_Y = 
general_X_train, general_X_test, general_y_train, general_y_test = train_test_split(general_X, general_y, test_size = TEST_SIZE, random_state=1)

In [ ]:
#create svm
print('h')

In [ ]:
#train svm with general reviews data set


In [ ]:
#Test accuracy of svm
